In [12]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re
import requests
import operator

In [45]:
class jmdPDFMap(object):
    """
    根据pdf生成文章，并计算推荐的准确率和召回率
    意图：导语，业绩变动原因，主营业务$业务进展，未来计划
    """
    def __init__(self):
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.prop = [0.2, 0.2, 0.4, 0.3, 0]
        self.K = 0
        #每个意图对应的句子或段落数量
        self.num = {}
        
        
    def computePR(self, article, pdfurl, mode = 'para'):
        intentList = []
        article_sen = self.get_article_sentence(article)
        article_len = len(article_sen)
        if article_len == 0:
            return -1, -1
        print 'article length: ' + str(article_len)
        if mode == 'para':
            pdftext = self.extractPDF(pdfurl, 0.8, ['para'])
        else:
            pdftext = self.extractPDF(pdfurl, 0.8, ['text'])
        for t in pdftext:
            intentList.append(self.getIntent(t))
        if mode == 'para':
            self.K = int(len(article_sen) * 0.5)
        else:
            self.K = int(len(article_sen) * 1.5)
        for l in range(len(self.labels)):
            self.num[self.labels[l]] = int(self.K * self.prop[l])
        recommendList = self.getTopKintent(intentList)
        rec_len = len(recommendList)
        if rec_len == 0:
            return -1, -1
        print 'recomment list length: ' + str(rec_len)
        P_dic = {}
        Pcount = 0
        Rcount = 0
        for r in recommendList:
            P_dic[r] = False
        for sen in article_sen:
            for rec in recommendList:
                if self.MatchTwoText(sen, rec):
                    Rcount += 1
                    if P_dic[rec] is False:
                        P_dic[rec] = True
                        Pcount += 1
        P = Pcount * 1.0 / rec_len
        R = Rcount * 1.0 / article_len
        print 'Pcount: ' + str(Pcount)
        print 'Rcount: ' + str(Rcount)
        return P, R
        
    def getTopKintent(self, intentlist):
        intent = {}
        for l in self.labels:
            intent[l] = {}
        
        for i in intentlist:
            key = i['intent']
            intent[key][i['content']] = i['prob']
        result = []
        for l in self.labels:
            intent_l = intent[l]
            intent[l] = sorted(intent_l.items(), key=operator.itemgetter(1))
        print "各个段落（句子）的预测意图分布为："
        for i in intent:
            print i + "  " + str(len(intent[i]))
        for l in intent.keys():
            result.extend([d[0] for d in intent[l][:self.num[l]]])
        return result
        
    def get_article_sentence(self, ar):
        """
        given an article, return the list of sentences
        """
        textList = ar.encode('utf-8').split('。') 
        textList = [t.replace('\n', '') for t in textList]
        textList = filter(lambda x: x != '', textList)
        return textList

    #percent: 取pdf的前多少页
    def extractPDF(self, pdfPath, percent, source = ['table', 'text', 'para']):
        savePath = '/home/zhuyuhe/mydata/jmd/percent/'
        ex = extractor.StockFinanceExtractor(pdfPath, percent)

        nodes = ex.get_pdf_objects(filter_headline=True)

        paraList = []
        textList = []
        textNodes = []
        tableList = []
        tableNodes = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
                continue
            if isinstance(node.obj, pdfextractor.page.Table):
                tableNodes.append(node)
            elif  not isinstance(node.obj, pdfextractor.page.Catalog):
                textNodes.append(node)
        if 'table' in source:
            tableList = self.get_node_texts(tableNodes)
            tableList = [t.replace('|', '').split('\n') for t in tableList]
            #二维数组转化为一维
            tableList = sum(tableList,[])
        textList = self.get_node_texts(textNodes)
        if 'para' in source:
            paraList = [t.replace('\n', '') for t in textList]
            paraList = filter(lambda x: x != '', paraList)
        elif 'text' in source:
            textList = sum([t.encode('utf-8').split('。') for t in textList],[])
            textList = [t.replace('\n', '') for t in textList]
            textList = filter(lambda x: x != '', textList)

        pdf = []
    #     pdf.append(textList)
    #     pdf.append(tableList)
    #     pdf = sum(pdf, [])
        pdf.extend(textList)
        pdf.extend(tableList)
        pdf.extend(paraList)
        print pdf.__len__()

        return pdf
    
    def get_node_texts(self, pdf_nodes):
        '''
        Given a list of nodes, get the texts of these nodes
        '''
        texts = []
        for node in pdf_nodes:
            text = node.obj.get_text()
            if len(text) <= 25:
                continue
            if u'□' in text or u'√' in text:
                continue
            texts.append(text)
        return texts
    
    def getIntent(self, text):
        """
        return intent of the given text
        """
        result = {}
        query = requests.post("https://surreal.aidigger.com/api/v1/classification/intent/finance/0",json={ "query": text}).json()
        predict = query['predict']
        prob = [x['prob'] for x in predict]
        i = max(prob)
        intent = predict[prob.index(i)]['name']
        #print intent
        result['content'] = text
        result['intent'] = self.intentMap(intent)
        result['prob'] = i
        return result
    
    def intentMap(self, i):
        """
        将现有意图映射到芥末堆所给的意图上去
        """
        i = i.encode('utf-8')
        if i == '意图_企业业绩__现状偏好' or i == '意图_企业业绩__现状偏坏' or i == '导语' or i == '意图_企业基本信息介绍_企业基本信息介绍':
            return self.labels[0]
        elif '原因' in i:
            return self.labels[1]
        elif '业务' in i and not '前景' in i:
            return self.labels[2]
        elif '前景' in i or '评论' in i:
            return self.labels[3]
        else:
            return self.labels[4]
        
    def MatchTwoText(self, t1, t2, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            return True
        return False


In [ ]:
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
P_para = []
R_para = []
P_sen = []
R_sen = []
i = 0
for jmap in jmdata_mapping:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1
#print a.getIntent('前三季度利润为0，同比下降5%')['intent']

# jmap = jmd_mapping[0]
# print a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')

******************************************************
article 0 :
--------------------
article length: 12
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 8s
Construct Pdf Tree time: 0s
44
各个段落（句子）的预测意图分布为：
主营业务$业务进展  4
导语  0
other  11
业绩变动原因  4
未来计划  3
recomment list length: 4
Pcount: 2
Rcount: 8
--PARA--  Pecision: 0.5    Recall: 0.666666666667
--------------------
article length: 12
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 9s
Construct Pdf Tree time: 0s
58
各个段落（句子）的预测意图分布为：
主营业务$业务进展  13
导语  1
other  18
业绩变动原因  19
未来计划  6
recomment list length: 16
Pcount: 3
Rcount: 3
--SEN--  Pecision: 0.1875    Recall: 0.25
---------------------
*******************************************************
******************************************************
article 1 :
--------------------
article length: 13
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 9s
Construct Pdf Tree time: 0s
14
各个段落（句子）的预测意图分布为：
主营业务$业务进

In [19]:
print a.get_article_sentence(jmap['content']).__len__()

12


In [44]:
P_para

[0.6666666666666666, -1, -1, 0.0, 1.0, 0.0]